### Part 3.2 - Model 3-English Google News 130GB corpus without OOV tokens

Set the version data control parameter (to save the outputs of this notebook at their latest date)

In [ ]:
version_data_control="22072020"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install those libraries if the notebook is executed on Google Colab

!pip install --quiet git+https://github.com/tensorflow/docs
!pip install --quiet humanfriendly
!pip install --quiet tqdm
!pip install --quiet unidecode

#### Import the libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import time
import random
import pickle
import json
import shutil
import unidecode

%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from datetime import datetime
from tabulate import tabulate
from packaging import version
from humanfriendly import format_timespan
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
date_format='%Y-%m-%d %H-%M-%S'

#### Improt visualization libraries

In [ ]:
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from pylab import rcParams

import pydot
import pydotplus
import graphviz

from IPython.display import SVG

#### Import Tensorflow

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_docs as tfdocs #!pip install git+https://github.com/tensorflow/docs
import tensorflow_docs.plots as tfplots
import tensorflow_docs.modeling as tfmodel

from tensorflow.keras import layers, regularizers, models
from tensorflow.keras import models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import model_to_dot, plot_model
from tensorflow.keras.models import load_model, model_from_json

#### Clear any logs from previous runs

In [ ]:
logging_directory=os.path.join(os.getcwd(), "model_three/logs/hparam_tuning")

In [ ]:
shutil.rmtree(logging_directory, ignore_errors=True)

#### Import Tensorboard

In [ ]:
%load_ext tensorboard
from tensorboard.plugins.hparams import api as hp

#### Import Tensorflow Hub

In [ ]:
import tensorflow_hub as hub

#### Import Keras

In [ ]:
import keras.backend as K
from tensorflow import keras

In [ ]:
%tensorflow_version 2.x

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

#### Import the dataset

In [ ]:
"""
Import the X_train, X_test, y_train & y_test data pickled from dataset part 3.1
"""
saved_version_data_control="13072020"
tokenization_history_folder="text_tokenization_padded_sequences"

X_train=pd.read_pickle(os.path.join(os.getcwd(), "drive/My Drive/data/{0}_{1}/X_train_all_inputs_{1}.pkl".format(tokenization_history_folder, saved_version_data_control)))
X_test=pd.read_pickle(os.path.join(os.getcwd(), "drive/My Drive/data/{0}_{1}/X_test_all_inputs_{1}.pkl".format(tokenization_history_folder, saved_version_data_control)))
y_train=pd.read_pickle(os.path.join(os.getcwd(), "drive/My Drive/data/{0}_{1}/y_train_all_inputs_{1}.pkl".format(tokenization_history_folder, saved_version_data_control)))
y_test=pd.read_pickle(os.path.join(os.getcwd(), "drive/My Drive/data/{0}_{1}/y_test_all_inputs_{1}.pkl".format(tokenization_history_folder, saved_version_data_control)))

assert X_train.shape[0]==y_train.shape[0]
assert X_test.shape[0]==y_test.shape[0]

In [ ]:
"""
Import genres
"""
with open(os.path.join(os.getcwd(), 'drive/My Drive/data/{0}_{1}/genres_list_06032020.pkl'.format(tokenization_history_folder, saved_version_data_control)),'rb') as f:
    genres_list = pickle.load(f)
genres_list

## <b>- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -  </b>

Create the data to be used by the pre-trained saved model. Each model expects the data of a specific format. The researcher should read the relative documentation to understand the proper data format per pre-trained save model.

In [ ]:
train_text_features = X_train['clean_combined_features'].tolist() #input 1
test_text_features = X_test['clean_combined_features'].tolist()

train_text_plot = X_train['clean_plot_summary'].tolist() #input 2
test_text_plot = X_test['clean_plot_summary'].tolist()

train_text_actors = X_train['clean_actors'].tolist() #input 3
test_text_actors = X_test['clean_actors'].tolist()

train_text_reviews = X_train['clean_reviews'].tolist() #input 4
test_text_reviews = X_test['clean_reviews'].tolist()

train_text_title = X_train['clean_movie_title'].tolist() #input 5
test_text_title = X_test['clean_movie_title'].tolist()

train_label = y_train.values
test_label = y_test.values

train_bytes_list_features = []
train_bytes_list_plot = []
train_bytes_list_actors = []
train_bytes_list_reviews = []
train_bytes_list_title = []

test_bytes_list_features = []
test_bytes_list_plot = []
test_bytes_list_actors = []
test_bytes_list_reviews = []
test_bytes_list_title = []

train_bytes_list_features=np.array([str.encode(unidecode.unidecode(i)) for i in train_text_features])
train_bytes_list_plot=np.array([str.encode(unidecode.unidecode(i)) for i in train_text_plot])
train_bytes_list_actors=np.array([list(map(lambda x: str.encode(unidecode.unidecode(x)), i.split(','))) for i in train_text_actors])
train_bytes_list_reviews=np.array([str.encode(unidecode.unidecode(i)) for i in train_text_reviews])
train_bytes_list_title=np.array([str.encode(unidecode.unidecode(i)) for i in train_text_title])

test_bytes_list_features=np.array([str.encode(unidecode.unidecode(i)) for i in test_text_features])
test_bytes_list_plot=np.array([str.encode(unidecode.unidecode(i)) for i in test_text_plot])
test_bytes_list_actors=np.array([list(map(lambda x: str.encode(unidecode.unidecode(x)), i.split(','))) for i in test_text_actors])
test_bytes_list_reviews=np.array([str.encode(unidecode.unidecode(i)) for i in test_text_reviews])
test_bytes_list_title=np.array([str.encode(unidecode.unidecode(i)) for i in test_text_title])

partial_x_train_features, x_val_features, partial_y_train, y_val = train_test_split(train_bytes_list_features, train_label, test_size=0.20, random_state=42)
partial_x_train_plot, x_val_plot, partial_y_train, y_val = train_test_split(train_bytes_list_plot, train_label, test_size=0.20, random_state=42)
partial_x_train_actors, x_val_actors, partial_y_train, y_val = train_test_split(train_bytes_list_actors, train_label, test_size=0.20, random_state=42)
partial_x_train_reviews, x_val_reviews, partial_y_train, y_val = train_test_split(train_bytes_list_reviews, train_label, test_size=0.20, random_state=42)
partial_x_train_title, x_val_title, partial_y_train, y_val = train_test_split(train_bytes_list_title, train_label, test_size=0.20, random_state=42)

assert partial_x_train_actors.shape[0]==partial_x_train_plot.shape[0]==partial_x_train_features.shape[0]==partial_x_train_reviews.shape[0]==partial_x_train_title.shape[0]==partial_y_train.shape[0]

In [ ]:
# Option 1: Fit the 5 inputs as expected by the TensorFlow Hub Model.

partial_x_train_features_option1=partial_x_train_features.astype(np.object)
partial_x_train_plot_option1=partial_x_train_plot.astype(np.object)
partial_x_train_reviews_option1=partial_x_train_reviews.astype(np.object)
partial_x_train_title_option1=partial_x_train_title.astype(np.object)
partial_x_train_actors_option1=np.asarray([b" ".join(i) for i in partial_x_train_actors]).astype(np.object)

test_bytes_list_features_option1=test_bytes_list_features.astype(np.object)
test_bytes_list_plot_option1=test_bytes_list_plot.astype(np.object)
test_bytes_list_reviews_option1=test_bytes_list_reviews.astype(np.object)
test_bytes_list_title_option1=test_bytes_list_title.astype(np.object)
test_bytes_list_actors_option1=np.asarray([b" ".join(i) for i in test_bytes_list_actors]).astype(np.object)

x_val_features_option1=partial_x_train_features.astype(np.object)
x_val_plot_option1=x_val_plot.astype(np.object)
x_val_reviews_option1=x_val_reviews.astype(np.object)
x_val_title_title_option1=x_val_title.astype(np.object)
x_val_actors_option1=np.asarray([b" ".join(i) for i in x_val_actors]).astype(np.object)

In [ ]:
print("Features", partial_x_train_features_option1[0:10], "\n")
print("Plot Summary", partial_x_train_plot_option1[0:10], "\n")
print("Reviews", partial_x_train_reviews_option1[0:10], "\n")
print("Movie Title", partial_x_train_title_option1[0:10], "\n")
print("Actors", partial_x_train_actors_option1[0:10], "\n")

## <b>- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -  </b>

#### Python Cell no.1
--------------------------

Define the functions that will be used accross the notebook. <br>
* visualise_model: This function is used to create a more informative image of the neural network structure.

* callback: The callbacks function with Early Stopping, Model checkpoints and Learning rate monitoring.

* save_model: Save the model's weights and structure at the end of the training

* plot_keras_history: Plot the learning curves of validation and training datasets across the training epochs

* hamming_loss: calculate the hamming loss value to evaluate the performance of the algorithm on test data. Not to be confused to the Hamming loss used as the performance metric.

In [ ]:
# Function 1 - Visualize Neural Network structure.

def visualize_model(model):
    return SVG(model_to_dot(model, show_shapes= True, show_layer_names=True, dpi=65).create(prog='dot', format='svg'))

#------------------------------------------#------------------------------------------

# Fucntion 2 - Callback function with early stopping to avod overfit.

def callback(mode, folder_path, saved_model_name, patience_value, logdir, hparams):
    
    # Initialize parameters
    monitor_metric = 'val_loss'
    minimum_delta = 0.009
    patience_limit = patience_value
    verbose_value = 1
    mode_value = 'min'
    weights_fname = os.path.join(os.getcwd(), '{0}/{1}.h5'.format(folder_path, saved_model_name))
    
    if mode == "step decay":
        # Initialize callbacks
        callbacks = [
            
            EarlyStopping(monitor=monitor_metric,
                          min_delta=minimum_delta,
                          patience=patience_limit,
                          verbose=verbose_value,
                          mode=mode_value,
                          restore_best_weights=True),

            ModelCheckpoint(filepath=weights_fname,
                            monitor=monitor_metric,
                            verbose=verbose_value,
                            save_best_only=True,
                            save_weights_only=True),
            
            tf.keras.callbacks.TensorBoard(logdir),
            
            hp.KerasCallback(logdir, hparams),

            tf.keras.callbacks.LearningRateScheduler(step_decay, verbose=1)
        ]

    else:
        # Initialize callbacks
        callbacks = [
            
            EarlyStopping(monitor=monitor_metric,
                          min_delta=minimum_delta,
                          patience=patience_limit,
                          verbose=verbose_value,
                          mode=mode_value,
                          restore_best_weights=True),

            ModelCheckpoint(filepath=weights_fname,
                            monitor=monitor_metric,
                            verbose=verbose_value,
                            save_best_only=True,
                            save_weights_only=True),
            
            tf.keras.callbacks.TensorBoard(logdir),
            
            hp.KerasCallback(logdir, hparams)
        ]

    return callbacks

#------------------------------------------#------------------------------------------

# Fucntion 3 - Save the neural network's weights and structure

def save_model(model, folder_path, model_name):
    
    model_json = model.to_json()

    with open(os.path.join(os.getcwd(), "{0}/{1}.json".format(folder_path, model_name)), "w") as json_file:
        json.dump(model_json, json_file)

    model.save_weights(os.path.join(os.getcwd(), "{0}/{1}.h5".format(folder_path, model_name)))
    
    print("\nModel's weights are saved")

#------------------------------------------#------------------------------------------

# Fucntion 4 - Plot the learning curves of the neural network

def plot_keras_history(history, folder_path, batch_size, lr_value, decaymiltiplier, version_data_control):
    
    metrics_names = [key for key in history.keys() if not key.startswith('val_')][0:3]
    
    fig = plt.gcf()
    fig, axs = plt.subplots(3, figsize=(20,18))
    
    for i, metric in enumerate(metrics_names):
        
        # getting the training values
        metric_train_values = history.get(metric, [])

        # getting the validation values
        metric_val_values = history.get("val_{0}".format(metric), [])

        # As loss always exists as a metric we use it to find the 
        epochs = range(1, len(metric_train_values) + 1)
        
        # leaving extra spaces to allign with the validation text
        training_text = "   Training {}: {:.5f}".format(metric,
                                                        metric_train_values[-1])
        print(training_text)

        axs[i].plot(epochs, 
                    metric_train_values,
                    'b',
                    label=training_text,
                    marker=".")
        
        axs[i].yaxis.label.set_color('white')
        axs[i].xaxis.label.set_color('white')
        
        axs[i].tick_params(axis='x', colors='white')
        axs[i].tick_params(axis='y', colors='white')
        
        axs[i].title.set_color('white')
        axs[i].spines['left'].set_color('white')
        axs[i].spines['bottom'].set_color('white')

        # if we validation metric exists, then plot that as well
        if metric_val_values:
            
            validation_text = "Validation {}: {:.5f}".format(metric,
                                                             metric_val_values[-1])

            axs[i].plot(epochs,
                        metric_val_values,
                        'g',
                        label=validation_text,
                        marker=".")
            
            axs[i].yaxis.label.set_color('white')
            axs[i].xaxis.label.set_color('white')

            axs[i].tick_params(axis='x', colors='white')
            axs[i].tick_params(axis='y', colors='white')

            axs[i].title.set_color('white')
            axs[i].spines['left'].set_color('white')
            axs[i].spines['bottom'].set_color('white')
        
        # add title, xlabel, ylabe, and legend
        axs[i].set_title('Model Metric: {}'.format(metric))
        axs[i].set_xlabel('Epochs')
        axs[i].set_ylabel(metric.title())
        axs[i].legend()
    
    fig.savefig(os.path.join(os.getcwd(), '{0}/ploting_training_validation_performance_{1}batchsize_{2}lr_{3}decaymultiplier_{4}.png'.format(folder_path, str(batch_size), str(lr_value), str(decaymiltiplier), version_data_control)), dpi=100)
    plt.show()
    plt.draw()
    plt.close()
            
#------------------------------------------#------------------------------------------

# Fucntion 5 - Calculate the Hamming loss of a multilabeled dataset

#source:https://github.com/tensorflow/addons/issues/305

def hamming_loss(y_true, y_pred, mode='multilabel'):
    if mode not in ['multiclass', 'multilabel']:
        raise TypeError('mode must be: [multiclass, multilabel])')

    if mode == 'multiclass':
        nonzero = tf.cast(tf.math.count_nonzero(y_true * y_pred, axis=-1), tf.float32)
        print("Hamming loss for multi-class classification is: ", 1.0-nonzero)
        return 1.0 - nonzero

    else:
        nonzero = tf.cast(tf.math.count_nonzero(y_true - y_pred, axis=-1), tf.float32)
        print("Hamming loss for multi-label classification is: ", nonzero / y_true.shape[-1] )
        return nonzero / y_true.shape[-1]

class HammingLoss(tfa.metrics.MeanMetricWrapper):
    def __init__(self, name='hamming_loss', dtype=None, mode='multilabel'):
        super(HammingLoss, self).__init__(
                hamming_loss, name, dtype=dtype, mode=mode)

#### Python Cell no.2
------------------------------

In this below python cell I keep track of the model parameters used to:

* create the neural network model,
* to fit the neural network,
* to optimize the neural network.

Storing the values of the parameters to a dictionary, I could then change dynamically the value of a parameter, rerun the neural model and then monitor the difference in the results.

**HyperParameters - Initialized**

In [ ]:
hp_logging_directory=os.path.join(os.getcwd(), "model_three/logs/hparam_tuning")

In [ ]:
HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([32, 64, 128]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001, 0.01, 0.1]))
HP_DECAY_STEPS_MULTIPLIER = hp.HParam('decay_steps_multiplier', hp.Discrete([10, 20]))

METRIC_ACCURACY = "hamming_loss"

In [ ]:
with tf.summary.create_file_writer(hp_logging_directory).as_default():
    hp.hparams_config(
    hparams=[HP_HIDDEN_UNITS, HP_LEARNING_RATE, HP_DECAY_STEPS_MULTIPLIER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='hamming_loss')],
  )
    
try:
    os.path.exists(hp_logging_directory)
    print("Directory of hyper parameters logging exists!")
except Exception as e:
    print(e)
    print("Directory not found!")

Directory of hyper parameters logging exists!


**Initialize the parameters non-tuned and the optimizers**

In [ ]:
neural_network_parameters = {}
fit_parameters = {}
optimizer_parameters = {}

#======================================================================
#           PARAMETERS THAT DEFINE THE NEURAL NETWORK STRUCTURE       =
#======================================================================

neural_network_parameters['l2_regularization'] = 0.01
neural_network_parameters['dropout_rate'] = 0.1
neural_network_parameters['dense_activation'] = 'relu'
neural_network_parameters['output_activation'] = 'sigmoid'
neural_network_parameters['number_target_variables'] = y_train.iloc[0].shape[-1]

neural_network_parameters['model_loss'] = tf.keras.losses.BinaryCrossentropy(from_logits=False, name='binary_crossentropy')

neural_network_parameters['model_metric'] = [tfa.metrics.HammingLoss(mode="multilabel", name="hamming_loss"), 
                                             tfa.metrics.F1Score(y_train.iloc[0].shape[-1], average="micro", name="f1_score_micro"), 
                                             tfa.metrics.F1Score(y_train.iloc[0].shape[-1], average=None, name="f1_score_none"),
                                             tfa.metrics.F1Score(y_train.iloc[0].shape[-1], average="macro", name="f1_score_macro")]

#---------------------------------------------------------------------------------------

#===================================================
#       PARAMETERS THAT DEFINE EACH OPTIMIZER      =
#===================================================

# Define a function to monitor the learning rate per epoch (Option 1)
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr

# learning rate schedule ((Option 2)
def step_decay(epoch, hparams):
    initial_lrate = hparams[HP_LEARNING_RATE]
    drop = 0.5
    epochs_drop = 20.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate
    
# Optimizer: ADAM (Learning scheduler with Inverse Time Decay)

optimizer_parameters['lr_scheduler_decay_rate'] = 0.1
optimizer_parameters['staircase'] = False
optimizer_parameters['validation_split_ratio']=0.7

def optimizer_adam_v2(haparms):
    
    print("Decay Steps of Inverse Time Decay: {0}".format(int(np.ceil((X_train.shape[0]*optimizer_parameters['validation_split_ratio'])//hparams[HP_HIDDEN_UNITS]))*hparams[HP_DECAY_STEPS_MULTIPLIER]))
    
    return keras.optimizers.Adam(tf.keras.optimizers.schedules.InverseTimeDecay(
        initial_learning_rate=hparams[HP_LEARNING_RATE],
        decay_steps=int(np.ceil((X_train.shape[0]*optimizer_parameters['validation_split_ratio'])//hparams[HP_HIDDEN_UNITS]))*hparams[HP_DECAY_STEPS_MULTIPLIER],
        decay_rate=optimizer_parameters['lr_scheduler_decay_rate'],
        staircase=optimizer_parameters['staircase']))

#---------------------------------------------------------------------------------------

#==================================================
#       PARAMETERS THAT DEFINE THE MODEL FIT      =
#==================================================

fit_parameters["epoch"] = 150
fit_parameters["patience_value"] = 20
fit_parameters["verbose_fit"] = 1
fit_parameters['validation_data_ratio']=1-optimizer_parameters['validation_split_ratio']
fit_parameters["cv_enabled"]="0"
fit_parameters["option_method"]="option1"

#### Python Cell no.3
--------------------------

The function creates the neural network structure, and fits the model on the dataset.

In [ ]:
# Import the pre-trained saved model from the relevant link of Tensorflow HUB

saved_model_url = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(saved_model_url, output_shape=[20], input_shape=[], dtype=tf.string, trainable=True)

# Initialize the basic folder path names that outputs
sequential_model_name="English_Google_News_130GB_20dim_without_OOV"
network_structure_file_name="network_structure_English_Google_News_130GB_without_OOV"
folder_path_model_saved="drive/My Drive/model_three/english_google_news_20dim_no_OOV"
saved_model_name="English_Google_News_130GB_20dim_without_OOV"
saved_metrics_dataframe_name="metrics_histogram_English_Google_News_130GB_20dim_without_OOV"
saved_df_scored_metric_name="df_metrics_english_google_news_130GB_20dim_without_oov"

In [ ]:
def create_fit_tf_hub_model(hparams,
                            version_data_control,
                            optimizer_name,
                            option_method,
                            callbacks,
                            optimizer_version = None):

    model = tf.keras.Sequential(name="{0}_{1}batchsize_{2}lr_{3}decaymultiplier_{4}".format(sequential_model_name,
                                                                                            str(hparams[HP_HIDDEN_UNITS]),
                                                                                            str(hparams[HP_LEARNING_RATE]),
                                                                                            str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                            version_data_control))
    model.add(hub_layer)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(hparams[HP_HIDDEN_UNITS],
                                    kernel_regularizer=regularizers.l2(neural_network_parameters['l2_regularization']),
                                    activation=neural_network_parameters['dense_activation']))
    model.add(tf.keras.layers.Dropout(neural_network_parameters['dropout_rate']))
    model.add(tf.keras.layers.Dense(y_val.shape[1], activation=neural_network_parameters['output_activation']))

    print(model.summary())

    #instantiate Optimizer
    if optimizer_name=="adam" and optimizer_version is None:

        optimizer = optimizer_adam_v2(hparams)

    model.compile(optimizer=optimizer, #else put "adam"
                  loss=neural_network_parameters['model_loss'],
                  metrics=[neural_network_parameters['model_metric']])
    
    #plot model's structure
    plot_model(model, show_shapes=True, dpi=70, to_file=os.path.join(os.getcwd(), '{0}/{1}_{2}batchsize_{3}lr_{4}decaymultiplier_{5}.png'.format(folder_path_model_saved,
                                                                                                                                                  network_structure_file_name, 
                                                                                                                                                  str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                                                  str(hparams[HP_LEARNING_RATE]), 
                                                                                                                                                  str(hparams[HP_DECAY_STEPS_MULTIPLIER]), 
                                                                                                                                                  version_data_control)))
    start_time = time.time()
    
    if option_method=="option1": # Use only this option. The preprocessing for options 2.1, 2.2 is deleted
        fitted_model=model.fit([partial_x_train_plot_option1, 
                                partial_x_train_features_option1,
                                partial_x_train_reviews_option1,
                                partial_x_train_title_option1],
                               partial_y_train,
                               steps_per_epoch=int(np.ceil((X_train.shape[0]*optimizer_parameters['validation_split_ratio'])//hparams[HP_HIDDEN_UNITS])),
                               epochs=fit_parameters["epoch"],
                               verbose=fit_parameters["verbose_fit"],
                               batch_size=hparams[HP_HIDDEN_UNITS],
                               validation_split=fit_parameters['validation_data_ratio'],
                               callbacks=callbacks)
        evaluation = model.evaluate([test_bytes_list_plot_option1,
                                    test_bytes_list_features_option1, 
                                    test_bytes_list_reviews_option1, 
                                    test_bytes_list_title_option1], test_label, batch_size=hparams[HP_HIDDEN_UNITS], verbose=2)

    elif option_method=="option2.1":
        fitted_model=model.fit(dataset.repeat().batch(hparams[HP_HIDDEN_UNITS], drop_remainder=True).prefetch(1),
                               steps_per_epoch=int(np.ceil(len(partial_x_train_actors_option2)//hparams[HP_HIDDEN_UNITS])),
                               epochs=fit_parameters["epoch"],
                               verbose=fit_parameters["verbose_fit"],
                               batch_size=hparams[HP_HIDDEN_UNITS],
                               validation_data=dataset_validation,
                               validation_steps=int(np.ceil(len(x_val_features_option2)//hparams[HP_HIDDEN_UNITS])),
                               callbacks=callbacks)
        evaluation = model.evaluate(dataset_test, batch_size=hparams[HP_HIDDEN_UNITS], verbose=2)

    else:
        fitted_model=model.fit([partial_x_train_actors_tensor,
                                partial_x_train_plot_tensor,
                                partial_x_train_features_tensor,
                                partial_x_train_reviews_tensor,
                                partial_x_train_title_tensor],
                               partial_y_train,
                               steps_per_epoch=int(np.ceil((X_train.shape[0]*optimizer_parameters['validation_split_ratio'])//hparams[HP_HIDDEN_UNITS])),
                               epochs=fit_parameters["epoch"],
                               verbose=fit_parameters["verbose_fit"],
                               batch_size=hparams[HP_HIDDEN_UNITS],
                               validation_data=([x_val_actors_tensor, x_val_plot_tensor, x_val_features_tensor, x_val_reviews_tensor, x_val_title_tensor],
                                                y_val),
                               callbacks=callbacks)
        evaluation = model.evaluate([test_bytes_list_actors_tensor, 
                                    test_bytes_list_plot_tensor,
                                    test_bytes_list_features_tensor, 
                                    test_bytes_list_reviews_tensor, 
                                    test_bytes_list_title_tensor], test_label, batch_size=hparams[HP_HIDDEN_UNITS], verbose=2)
    #save the model
    save_model(model,
               folder_path_model_saved,
               "{0}_{1}batchsize_{2}lr_{3}decaymultiplier_{4}".format(saved_model_name,
                                                                      str(hparams[HP_HIDDEN_UNITS]),
                                                                      str(hparams[HP_LEARNING_RATE]),
                                                                      str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                      version_data_control))
    elapsed_time = time.time() - start_time
    
    print("\nTraining time of the multi-input keras model has finished. Duration {0} secs".format(format_timespan(elapsed_time)))

    loss = evaluation[0] # single number
    model_metric = evaluation[1:] # is a list of 3 elements (hamming loss, f1_score_micro, f1_score_none)

    hamming_loss_value=evaluation[1]
    print('Hamming loss value on test data: {0}'.format(hamming_loss_value))

    return hamming_loss_value, model, fitted_model

#### Python Cell no.4
--------------------------

*run* is a method that call the above function *create_fit_keras_model*

In [ ]:
def run(run_dir, hparams, version_data_control, optimizer_name, option_method, callbacks):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        accuracy, model, fitted_model = create_fit_tf_hub_model(hparams, version_data_control, optimizer_name, option_method, callbacks)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
    
    return model, fitted_model

Everything is set. In the next coding cells the training is executed! <br>
<b>- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -  </b>

#### Python Cell no.5
--------------------------

Fit the model and monitor the performance of the classifiers

In [ ]:
if os.path.exists(os.path.join(os.getcwd(), "{0}".format(folder_path_model_saved))) is True:
    print("Folder already exists!\n")
else:
    print("Folder not found!\n")
    os.mkdir(os.path.join(os.getcwd(), "{0}".format(folder_path_model_saved)))
    print("Folder is created!\n")

In [ ]:
"""
Model Training
"""
begin_time=time.time()
print("{0}: Start execution of the cell\n".format(datetime.utcnow().strftime(date_format)))

session_num = 1

for batch_size in HP_HIDDEN_UNITS.domain.values:
  for learning_rate in HP_LEARNING_RATE.domain.values:
      for decay_steps_multiplier in HP_DECAY_STEPS_MULTIPLIER.domain.values:
          hparams = {
              HP_HIDDEN_UNITS: batch_size,
              HP_LEARNING_RATE: learning_rate,
              HP_DECAY_STEPS_MULTIPLIER: decay_steps_multiplier
            }
          run_name = "run-id {0}".format(session_num)
          total_number_models=(len(HP_HIDDEN_UNITS.domain.values)*len(HP_LEARNING_RATE.domain.values)*len(HP_DECAY_STEPS_MULTIPLIER.domain.values))
          print('--- Starting trial: {0}/{1}\n'.format(run_name, total_number_models))
          print({h.name: hparams[h] for h in hparams}, '\n')

          starting_training=time.time()

          model_struture, model_history=run('{0}/'.format(hp_logging_directory) + run_name,
                                            hparams,
                                            version_data_control,
                                            "adam",
                                            fit_parameters["option_method"],
                                            callback("adam-inverse decay",
                                                      folder_path_model_saved,
                                                      "{0}_{1}batchsize_{2}lr_{3}decaymultiplier_{4}".format(saved_model_name,
                                                                                                            str(batch_size),
                                                                                                            str(learning_rate),
                                                                                                            str(decay_steps_multiplier),
                                                                                                            version_data_control),
                                                      fit_parameters["patience_value"],
                                                      "{0}/".format(hp_logging_directory) + datetime.now().strftime("%Y%m%d-%H%M%S"), 
                                                      hparams))
          
          print("Average time per epoch: {0}\n".format(format_timespan((time.time()-starting_training)/len(model_history.epoch))))
          
          hist = pd.DataFrame(model_history.history)
          hist['epoch'] = model_history.epoch
          hist['epoch']+= 1
          hist.index += 1
          print("Table of training the {0} text classification model\n".format(sequential_model_name))
          print(tabulate(hist, headers='keys', tablefmt='psql'))

          hist.to_pickle(os.path.join(os.getcwd(), folder_path_model_saved+"/{0}_{1}batchsize_{2}lr_{3}decaymultiplier_{4}.pkl".format(saved_metrics_dataframe_name,
                                                                                                                                       str(batch_size), 
                                                                                                                                       str(learning_rate), 
                                                                                                                                       str(decay_steps_multiplier), 
                                                                                                                                       version_data_control)))

          #plot the model's model_metric (Hamming Loss, F1-score) & loss
          plot_keras_history(model_history.history, folder_path_model_saved, batch_size, learning_rate, decay_steps_multiplier, version_data_control)

          #evaluate the model
          if fit_parameters["option_method"]=="option1":
              model_evaluation = model_struture.evaluate([test_bytes_list_plot_option1,
                                                          test_bytes_list_features_option1,
                                                          test_bytes_list_reviews_option1,
                                                          test_bytes_list_title_option1],
                                                         test_label,
                                                         batch_size=batch_size,
                                                         verbose=2)
              
          elif fit_parameters["option_method"]=="option2.1":
              model_evaluation = model_struture.evaluate(dataset_test, batch_size=hparams[HP_HIDDEN_UNITS], verbose=2)
          
          else:
              model_evaluation = model_struture.evaluate([test_bytes_list_actors_tensor, 
                                                          test_bytes_list_plot_tensor,
                                                          test_bytes_list_features_tensor, 
                                                          test_bytes_list_reviews_tensor, 
                                                          test_bytes_list_title_tensor], 
                                                         test_label, 
                                                         batch_size=hparams[HP_HIDDEN_UNITS], 
                                                         verbose=2)

          print("\nTest Score (evalution of the model's loss/error on the test sequences): {0}".format(model_evaluation[0]))
          print("\nTest model_metric (evalution of the hamming loss on the test sequences): {0}\n".format(model_evaluation[1]))

          fig = plt.figure()
          ax = fig.add_subplot(111)
          
          plt.rcParams["figure.figsize"] = (16,13)
          
          ax.bar(genres_list, model_evaluation[3])
          
          ax.set_title('F1 score per genre tag')
          ax.set_xlabel('Movie Genre')
          ax.set_ylabel('F1 score')
          ax.set_xticklabels(genres_list)
          
          ax.spines['left'].set_color('white')
          ax.spines['bottom'].set_color('white')

          ax.tick_params(axis='x', colors='white')
          ax.tick_params(axis='y', colors='white')
          
          ax.yaxis.label.set_color('white')
          ax.xaxis.label.set_color('white')
          ax.title.set_color('white')

          plt.xticks(rotation=45, ha='right')

          for i, f1 in enumerate(model_evaluation[3]):
              ax.annotate(round(f1, 2), (i, f1), ha='center', va='bottom')
          
          plt.show()

          session_num += 1

total_time=time.time() - begin_time
print("{0}: Total cell execution time: {1}".format(datetime.utcnow().strftime(date_format), format_timespan(total_time)))

#### - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

#### Python Cell no.6
--------------------------

Select the best model classifier

In [ ]:
saved_model_name="English_Google_News_130GB_20dim_without_OOV"
folder_path_model_saved="model_three/english_google_news_20dim_no_OOV"
saved_df_scored_metric_name="df_metrics_English_Google_News_130GB_20dim_without_OOV"

In [ ]:
# Function 1

def import_trained_keras_model(method, decay_steps_mode, embedding_dim_mode, optimizer_name, hparams):
    """
    Load the weights of the model saved with EarlyStopping
    """
    if method == "import custom trained model":
        
        if decay_steps_mode=="on":
            
            if embedding_dim_mode=="on":
            
                with open(os.path.join(os.getcwd(), "{0}/{1}_{2}dim_{3}batchsize_{4}lr_{5}decaymultiplier_{6}.json".format(folder_path_model_saved,
                                                                                                                            saved_model_name,
                                                                                                                            str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                                            str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                            str(hparams[HP_LEARNING_RATE]), 
                                                                                                                            str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                            version_data_control)),'r') as f:
                    model_json = json.load(f)

                model_imported = model_from_json(model_json)

                model_imported.load_weights(os.path.join(os.getcwd(), '{0}/{1}_{2}dim_{3}batchsize_{4}lr_{5}decaymultiplier_{6}.h5'.format(folder_path_model_saved,
                                                                                                                                            saved_model_name,
                                                                                                                                            str(hparams[HP_EMBEDDING_DIM]),
                                                                                                                                            str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                                            str(hparams[HP_LEARNING_RATE]), 
                                                                                                                                            str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                                            version_data_control)))
            else:
                
                with open(os.path.join(os.getcwd(), "{0}/{1}_{2}batchsize_{3}lr_{4}decaymultiplier_{5}.json".format(folder_path_model_saved,
                                                                                                                     saved_model_name,
                                                                                                                     str(hparams[HP_HIDDEN_UNITS]),
                                                                                                                     str(hparams[HP_LEARNING_RATE]),
                                                                                                                     str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                     version_data_control)),'r') as f:
                    model_json = json.load(f)

                model_imported = model_from_json(model_json)

                model_imported.load_weights(os.path.join(os.getcwd(), '{0}/{1}_{2}batchsize_{3}lr_{4}decaymultiplier_{5}.h5'.format(folder_path_model_saved,
                                                                                                                                            saved_model_name,
                                                                                                                                            str(hparams[HP_HIDDEN_UNITS]),
                                                                                                                                            str(hparams[HP_LEARNING_RATE]), 
                                                                                                                                            str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                                            version_data_control)))
        else:
            
            with open(os.path.join(os.getcwd(), "{0}/{1}_{2}dim_{3}batchsize_{4}lr_{5}.json".format(folder_path_model_saved,
                                                                                                     saved_model_name,
                                                                                                     str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                     str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                     str(hparams[HP_LEARNING_RATE]),
                                                                                                     version_data_control)),'r') as f:
                model_json = json.load(f)

            model_imported = model_from_json(model_json)

            model_imported.load_weights(os.path.join(os.getcwd(), '{0}/{1}_{2}dim_{3}batchsize_{4}lr_{5}.h5'.format(folder_path_model_saved,
                                                                                                                     saved_model_name,
                                                                                                                     str(hparams[HP_EMBEDDING_DIM]), 
                                                                                                                     str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                     str(hparams[HP_LEARNING_RATE]),
                                                                                                                     version_data_control))) 
        if optimizer_name=="adam":
            optimizer = optimizer_adam_v2(hparams)
        
        elif optimizer_name=="sgd":
            optimizer = optimizer_sgd_v1(hparams, "step decay")
            
        else:
            optimizer = optimizer_rmsprop_v1(hparams)
            
        model_imported.compile(optimizer=optimizer,
                               loss=neural_network_parameters['model_loss'],
                               metrics=neural_network_parameters['model_metric'])
        print("\nModel is loaded successfully\n")
    
    else:
        
        with open(os.path.join(os.getcwd(), "{0}/{1}_{2}batchsize_{3}lr_{4}decaymultiplier_{5}.json".format(folder_path_model_saved,
                                                                                                             saved_model_name,
                                                                                                             str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                             str(hparams[HP_LEARNING_RATE]), 
                                                                                                             str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                             version_data_control)),'r') as f:
            model_json = json.load(f)

        model_imported = model_from_json(model_json, custom_objects={'KerasLayer':hub.KerasLayer})

        model_imported.load_weights(os.path.join(os.getcwd(), '{0}/{1}_{2}batchsize_{3}lr_{4}decaymultiplier_{5}.h5'.format(folder_path_model_saved,
                                                                                                                             saved_model_name,
                                                                                                                             str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                             str(hparams[HP_LEARNING_RATE]), 
                                                                                                                             str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                             version_data_control)))

        optimizer = optimizer_adam_v2(hparams)

        model_imported.compile(optimizer=optimizer,
                               loss=neural_network_parameters['model_loss'],
                               metrics=neural_network_parameters['model_metric'])
        print("\nModel is loaded successfully\n")
    
    return model_imported

#----------------------------------------------------------------------

# Function 2

def create_df_scoring_table(method, decay_steps_mode, embedding_dim_mode, model_tag, hparams, model):
    """
    Create a scoring dictionary to select the best out of the four models
    """
    if method == "import custom trained model":
        
        model_evaluation = model.evaluate([X_test_seq_actors, X_test_seq_plot, X_test_seq_features, X_test_seq_reviews, X_test_seq_title],
                                          y_test,
                                          batch_size=hparams[HP_HIDDEN_UNITS],
                                          verbose=2)

        y_test_pred_probs = model.predict([X_test_seq_actors, X_test_seq_plot, X_test_seq_features, X_test_seq_reviews, X_test_seq_title])
        y_test_predictions = (y_test_pred_probs>0.5).astype(int)

        variance = np.var(y_test_predictions)
        sse = np.mean((np.mean(y_test_predictions) - y_test)**2)
        bias = sse - variance

        hamming_loss_value = HammingLoss(mode='multilabel')
        hamming_loss_value.update_state(y_test, y_test_predictions)
        
        if decay_steps_mode=="on":
            
            if embedding_dim_mode=="on":
            
                df_scores=pd.DataFrame({'Tag Name':pd.Series(model_tag, dtype='str'),
                                        'Embedding Dimension tag':pd.Series(hparams[HP_EMBEDDING_DIM], dtype='int'),
                                        'Batch tag':pd.Series(hparams[HP_HIDDEN_UNITS], dtype='int'),
                                        'Learning Rate tag':pd.Series(hparams[HP_LEARNING_RATE], dtype='float'),
                                        'Decay Multiplier tag':pd.Series(hparams[HP_DECAY_STEPS_MULTIPLIER], dtype='int'),
                                        'Test Loss':pd.Series([model_evaluation[0]], dtype='float'),
                                        'Test Hamming Loss':pd.Series([model_evaluation[1]], dtype='float'),
                                        'Hamming Loss':pd.Series([hamming_loss_value.result().numpy()], dtype='float'),
                                        'Zero_one Loss':pd.Series([zero_one_loss(y_test, y_test_predictions, normalize=False)], dtype='float'),
                                        'F1_score':pd.Series([f1_score(y_test, y_test_predictions, average="micro")], dtype='float'),
                                        'F1_score_samples':pd.Series([f1_score(y_test, y_test_predictions, average="samples")], dtype='float'),
                                        'ROC_score':pd.Series([roc_auc_score(y_test, y_test_predictions, average="micro", multi_class="ovr")], dtype='float'),
                                        'ROC_score_samples':pd.Series([roc_auc_score(y_test, y_test_predictions, average="samples", multi_class="ovr")], dtype='float'),
                                        'Bias':pd.Series([bias], dtype='float'),
                                        'Variance':pd.Series([variance], dtype='float')
                                       })

                df_scores.to_pickle(os.path.join(os.getcwd(), "{0}/{1}_{2}dim_{3}batchsize_{4}lr_{5}decaymultiplier_{6}.pkl".format(folder_path_model_saved,
                                                                                                                                     saved_df_scored_metric_name,
                                                                                                                                     str(hparams[HP_EMBEDDING_DIM]),
                                                                                                                                     str(hparams[HP_HIDDEN_UNITS]),
                                                                                                                                     str(hparams[HP_LEARNING_RATE]),
                                                                                                                                     str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                                     version_data_control)))
            else:
                
                df_scores=pd.DataFrame({'Tag Name':pd.Series(model_tag, dtype='str'),
                                        'Batch tag':pd.Series(hparams[HP_HIDDEN_UNITS], dtype='int'),
                                        'Learning Rate tag':pd.Series(hparams[HP_LEARNING_RATE], dtype='float'),
                                        'Decay Multiplier tag':pd.Series(hparams[HP_DECAY_STEPS_MULTIPLIER], dtype='int'),
                                        'Test Loss':pd.Series([model_evaluation[0]], dtype='float'),
                                        'Test Hamming Loss':pd.Series([model_evaluation[1]], dtype='float'),
                                        'Hamming Loss':pd.Series([hamming_loss_value.result().numpy()], dtype='float'),
                                        'Zero_one Loss':pd.Series([zero_one_loss(y_test, y_test_predictions, normalize=False)], dtype='float'),
                                        'F1_score':pd.Series([f1_score(y_test, y_test_predictions, average="micro")], dtype='float'),
                                        'F1_score_samples':pd.Series([f1_score(y_test, y_test_predictions, average="samples")], dtype='float'),
                                        'ROC_score':pd.Series([roc_auc_score(y_test, y_test_predictions, average="micro", multi_class="ovr")], dtype='float'),
                                        'ROC_score_samples':pd.Series([roc_auc_score(y_test, y_test_predictions, average="samples", multi_class="ovr")], dtype='float'),
                                        'Bias':pd.Series([bias], dtype='float'),
                                        'Variance':pd.Series([variance], dtype='float')
                                       })

                df_scores.to_pickle(os.path.join(os.getcwd(), "{0}/{1}_{2}batchsize_{3}lr_{4}decaymultiplier_{5}.pkl".format(folder_path_model_saved,
                                                                                                                              saved_df_scored_metric_name,
                                                                                                                              str(hparams[HP_HIDDEN_UNITS]),
                                                                                                                              str(hparams[HP_LEARNING_RATE]),
                                                                                                                              str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                              version_data_control)))    
        else:
            
            df_scores=pd.DataFrame({'Tag Name':pd.Series(model_tag, dtype='str'),
                                    'Embedding Dimension tag':pd.Series(hparams[HP_EMBEDDING_DIM], dtype='int'),
                                    'Batch tag':pd.Series(hparams[HP_HIDDEN_UNITS], dtype='int'),
                                    'Learning Rate tag':pd.Series(hparams[HP_LEARNING_RATE], dtype='float'),
                                    'Test Loss':pd.Series([model_evaluation[0]], dtype='float'),
                                    'Test Hamming Loss':pd.Series([model_evaluation[1]], dtype='float'),
                                    'Hamming Loss':pd.Series([hamming_loss_value.result().numpy()], dtype='float'),
                                    'Zero_one Loss':pd.Series([zero_one_loss(y_test, y_test_predictions, normalize=False)], dtype='float'),
                                    'F1_score':pd.Series([f1_score(y_test, y_test_predictions, average="micro")], dtype='float'),
                                    'F1_score_samples':pd.Series([f1_score(y_test, y_test_predictions, average="samples")], dtype='float'),
                                    'ROC_score':pd.Series([roc_auc_score(y_test, y_test_predictions, average="micro", multi_class="ovr")], dtype='float'),
                                    'ROC_score_samples':pd.Series([roc_auc_score(y_test, y_test_predictions, average="samples", multi_class="ovr")], dtype='float'),
                                    'Bias':pd.Series([bias], dtype='float'),
                                    'Variance':pd.Series([variance], dtype='float')
                                   })

            df_scores.to_pickle(os.path.join(os.getcwd(), "{0}/{1}_{2}dim_{3}batchsize_{4}lr_{5}.pkl".format(folder_path_model_saved,
                                                                                                              saved_df_scored_metric_name,
                                                                                                              str(hparams[HP_EMBEDDING_DIM]),
                                                                                                              str(hparams[HP_HIDDEN_UNITS]),
                                                                                                              str(hparams[HP_LEARNING_RATE]),
                                                                                                              version_data_control)))
    else:
        
        model_evaluation = model.evaluate([test_bytes_list_plot_option1, test_bytes_list_features_option1, test_bytes_list_reviews_option1, test_bytes_list_title_option1],
                                          test_label,
                                          batch_size=hparams[HP_HIDDEN_UNITS],
                                          verbose=2)

        y_test_pred_probs = model.predict([test_bytes_list_plot_option1, test_bytes_list_features_option1, test_bytes_list_reviews_option1, test_bytes_list_title_option1])
        y_test_predictions = (y_test_pred_probs>0.5).astype(int)

        variance = np.var(y_test_predictions)
        sse = np.mean((np.mean(y_test_predictions) - test_label)**2)
        bias = sse - variance

        hamming_loss_value = HammingLoss(mode='multilabel')
        hamming_loss_value.update_state(test_label, y_test_predictions)

        df_scores=pd.DataFrame({'Tag Name':pd.Series(model_tag, dtype='str'),
                                'Batch tag':pd.Series(hparams[HP_HIDDEN_UNITS], dtype='int'),
                                'Learning Rate tag':pd.Series(hparams[HP_LEARNING_RATE], dtype='float'),
                                'Decay Multiplier tag':pd.Series(hparams[HP_DECAY_STEPS_MULTIPLIER], dtype='int'),
                                'Test Loss':pd.Series([model_evaluation[0]], dtype='float'),
                                'Test Hamming Loss':pd.Series([model_evaluation[1]], dtype='float'),
                                'Hamming Loss':pd.Series([hamming_loss_value.result().numpy()], dtype='float'),
                                'Zero_one Loss':pd.Series([zero_one_loss(test_label, y_test_predictions, normalize=False)], dtype='float'),
                                'F1_score':pd.Series([f1_score(test_label, y_test_predictions, average="micro")], dtype='float'),
                                'F1_score_samples':pd.Series([f1_score(test_label, y_test_predictions, average="samples")], dtype='float'),
                                'ROC_score':pd.Series([roc_auc_score(test_label, y_test_predictions, average="micro", multi_class="ovr")], dtype='float'),
                                'ROC_score_samples':pd.Series([roc_auc_score(test_label, y_test_predictions, average="samples", multi_class="ovr")], dtype='float'),
                                'Bias':pd.Series([bias], dtype='float'),
                                'Variance':pd.Series([variance], dtype='float')
                               })

        df_scores.to_pickle(os.path.join(os.getcwd(), "{0}/{1}_{2}batchsize_{3}lr_{4}decaymultiplier_{5}.pkl".format(folder_path_model_saved,
                                                                                                                      saved_df_scored_metric_name,
                                                                                                                      str(hparams[HP_HIDDEN_UNITS]), 
                                                                                                                      str(hparams[HP_LEARNING_RATE]), 
                                                                                                                      str(hparams[HP_DECAY_STEPS_MULTIPLIER]),
                                                                                                                      version_data_control)))
    return df_scores

In [ ]:
model_method_creation="adam"

list_models=[]
list_df=[]

HP_HIDDEN_UNITS = hp.HParam('batch_size', hp.Discrete([32, 64, 128]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001, 0.01, 0.1]))
HP_DECAY_STEPS_MULTIPLIER = hp.HParam('decay_steps_multiplier', hp.Discrete([10, 20]))

for batch_size in HP_HIDDEN_UNITS.domain.values:
    for learning_rate in HP_LEARNING_RATE.domain.values:
        for decay_steps_multiplier in HP_DECAY_STEPS_MULTIPLIER.domain.values:
            hparams = {
                HP_HIDDEN_UNITS: batch_size,
                HP_LEARNING_RATE: learning_rate,
                HP_DECAY_STEPS_MULTIPLIER: decay_steps_multiplier
              }
            print("{0}/{1}".format(len(list_models)+1, (len(HP_HIDDEN_UNITS.domain.values)*len(HP_LEARNING_RATE.domain.values)*len(HP_DECAY_STEPS_MULTIPLIER.domain.values))))
            model_object=import_trained_keras_model("import pre-trained hub model", "on", "off", model_method_creation, hparams)
            df_object=create_df_scoring_table("import pre-trained hub model", "on", "off", "{0}-{1}".format(saved_model_name, len(list_models)+1), hparams, model_object)
            list_models.append(model_object)
            list_df.append(df_object)

In [ ]:
model_one, model_two, model_three, model_four, model_five, model_six, model_seven, model_eight, model_nine, model_ten, model_eleven, model_twelve, model_thirteen, model_fourteen, model_fifteen, model_sixteen, model_seventeen, model_eighteen=list_models

df_scores_one, df_scores_two, df_scores_three, df_scores_four, df_scores_five, df_scores_six, df_scores_seven, df_scores_eight, df_scores_nine, df_scores_ten, df_scores_eleven, df_scores_twelve, df_scores_thirteen, df_scores_fourteen, df_scores_fifteen, df_scores_sixteen, df_scores_seventeen, df_scores_eighteen=list_df

frames_20dim_noOOV=[df_scores_one, df_scores_two, df_scores_three, df_scores_four, df_scores_five, df_scores_six, df_scores_seven, df_scores_eight, df_scores_nine, df_scores_ten, df_scores_eleven, df_scores_twelve, df_scores_thirteen, 
                    df_scores_fourteen, df_scores_fifteen, df_scores_sixteen, df_scores_seventeen, df_scores_eighteen]

result_20dim_noOOV=pd.concat(frames_20dim_noOOV)
result_20dim_noOOV=result_20dim_noOOV.reset_index(drop=True)
result_20dim_noOOV.sort_values(by=['Hamming Loss', 'Zero_one Loss'])

Best model of the total 18 models presented above is model 3 with:
* Batch size: 32
* Learning rate: 0.01
* Decay Steps Multiplier: 10
* Hamming loss & Zeron-one loss: 0.092434 - 7761.0

In [ ]:
print(result_20dim_noOOV.to_latex(index=False))

In [ ]:
# Visualize the structure of the model selected
def visualize_model(model):
    return SVG(model_to_dot(model, show_shapes= True, show_layer_names=True, dpi=65).create(prog='dot', format='svg'))

visualize_model(model_three)